In [ ]:
!pip install SQLAlchemy==1.4.49
!pip install ipython-sql==0.4.1

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql://[ID]:[PASSWORD]@[ENDPOINT]

In [ ]:
%%sql

select table_name, table_type
from information_schema.tables
where table_schema ='analytics';

### Monthly Active User 차트

#### user_session_summary 테이블 생성


In [ ]:
%%sql

CREATE TABLE analytics.user_session_summary AS
  SELECT usc.*, t.ts
  FROM raw_data.user_session_channel usc
  LEFT JOIN raw_data.session_timestamp t ON t.sessionid = usc.sessionid;

#### mau_summary 테이블 생성

In [ ]:
%%sql

CREATE TABLE analytics.mau_summary AS
SELECT
  LEFT(ts, 7) "month",
  COUNT(DISTINCT userid) mau
FROM analytics.user_session_summary
GROUP BY 1
ORDER BY 1;

### Monthly Cohort 차트

#### cohort_summary 테이블 생성

In [ ]:
%%sql

CREATE TABLE analytics.cohort_summary AS
SELECT cohort_month, visited_month, cohort.userid
FROM(
  SELECT userid, date_trunc('month', MIN(ts)) cohort_month
  FROM analytics.user_session_summary
  GROUP BY 1
) cohort
JOIN (
  SELECT DISTINCT userid, date_trunc('month', ts) visited_month
  FROM analytics.user_session_summary
) visit ON cohort.cohort_month <= visit.visited_month and cohort.userid = visit.userid;

#### monthly_cohort 테이블 생성

In [ ]:
%%sql

CREATE TABLE analytics.monthly_cohort AS
SELECT
  DATEDIFF(month, cohort_month, visited_month) AS month,
  cohort_month,
  COUNT(userid) users
FROM analytics.cohort_summary
GROUP BY 1,2;

In [ ]:
%%sql

select *
from analytics.monthly_cohort
order by 1,2;